<a href="https://colab.research.google.com/github/vilsonrodrigues/PortfolioDataScience/blob/master/Modelo_de_Projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Por @vilsonrodrigues



# <center><font color='blue'>Nome do Projeto</font>




<img src='https://t8f8b3g9.stackpathcdn.com/wp-content/uploads/2019/07/3.png'>

## Parte I - Apresentação do Problema
### [1. Apresentação do Problema](#apresentacao)
## Parte II - Preparativos Iniciais
### [1. Bibliotecas](#bibliotecas)
### [2. Classe](#classe)
### [3. Funções](#funcoes)
### [4. Leitura da Base](#leiturabase)
## Parte III - Desenvolvimento
### [1. Entendimento de Negócio](#negocio)
### [2. Entendimento de Dados](#enten_dados)
### [3. Preparação de Dados](#preparo)
### [4. Métricas de Avaliação](#metricas)
### [5. Modelagem](#modelagem)
### [6. Avaliação de Modelo](#avaliacao)
### [7. Conclusão e Recomendações](#conclusao)
### [8. Implantação de Modelos](#implantacao)


# Parte I - Apresentação do Problema

### Meta

### Fonte dos Dados

### Referência
Hands On: Machine Learning with Scikit-Learning e Tensor Flow



# Parte II - Preparativos Iniciais

## 1. Bibliotecas
<a id='bibliotecas'></a>
#### [1.1 Instalações](#instalacao)
#### [1.2 Importações](#importacao)


### 1.1. Instalação
<a id='instalacao'></a>

In [0]:
!pip install pandas -U
!pip install scikit-learn -U
!pip install numpy -U
!pip install joblib==0.14.1
!conda install xgboost -c conda-forge --yes

### 1.2. Importação
<a id='importacao'></a>

In [0]:
import pandas as pd
import numpy as np

#Persistência em disco
import joblib
import pickle

#Visualização
import seaborn as sns
import matplotlib.pyplot as plt

#Transformadores e Modelador
from sklearn.pipeline          import Pipeline
from sklearn.preprocessing     import StandardScaler
from sklearn.base              import BaseEstimator, TransformerMixin

#Selecao de modelo
from sklearn.model_selection   import train_test_split
from sklearn.model_selection   import KFold
from sklearn.model_selection   import cross_val_score
from sklearn.model_selection   import GridSearchCV
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_selection import chi2

#Metricas
from sklearn.metrics           import classification_report
from sklearn.metrics           import confusion_matrix
from sklearn.metrics           import make_scorer
from sklearn.metrics           import accuracy_score
from sklearn.metrics           import f1_score
from sklearn.metrics           import recall_score
from sklearn.metrics           import precision_score

#Classificadores
from sklearn.ensemble          import RandomForestClassifier
from xgboost                   import XGBClassifier
from sklearn.tree              import DecisionTreeClassifier
from sklearn.neighbors         import KNeighborsClassifier
from sklearn.naive_bayes       import GaussianNB
from sklearn.svm               import SVC
from sklearn.neural_network    import MLPClassifier

#modo não-perturbe
import warnings
warnings.filterwarnings("ignore")

##2. Classe
<a id='classe'></a>


Classe para realizar x-ação

##3. Funções
<a id='funcoes'></a>

Função que usa GridSearch para avaliar o melhor modelo disponível, retornar estatísticas dos resultados e persistir em disco os modelos gerados

In [0]:
def avaliar_modelos(df, hyperparameters):

  #Dicionários
  model_param_disciplinas = {}
  best_acc_disciplinas = {}
  scoring_disciplinas = {}
  #feature_importances_disciplinas = {}

  #Seleciona o codigo de cada disciplina no dataframe, --alteravel--
  lista_disciplinas = df["disciplina"].unique()  

  #Verifica se esta vazio
  if lista_disciplinas is None:
	  return None

  #Convertendo categorias do target de string para numerico (0,1)
  df["situacao_categoria"] = pd.Categorical(df["situacao"]).codes

  #Pipeline para transformar em categoria e verificar melhores parametros
  pipeline_transform = Pipeline(steps = [('num_pipeline', NumericalTransformer()),
                                         ('fs',SelectKBest(chi2)),
                                         ('sc',StandardScaler()),
                                         ('clf',XGBClassifier())
                               ])

  #Grid para testar todos os parâmetros                               
  grid_search = GridSearchCV(
                          estimator = pipeline_transform, 
                          param_grid = hyperparameters,
                          cv= 5,
                          scoring = {'Accuracy': make_scorer(accuracy_score)},
                          #scoring = {"AUC": "roc_auc", "Accuracy": make_scorer(accuracy_score), "F1":"f1", "Recall":"recall", "Precision":"precision"},
                          return_train_score=True,
                          n_jobs=-1,#indica o numero de processos em paralelo, -1 significa usar todos
                          refit='Accuracy')


  #itera sobre a lista de disciplinas
  for disciplina in lista_disciplinas:
    data_iter = df.loc[df["disciplina"].str.contains(disciplina),:]
    
    #Treinando Grid
    best_model = grid_search.fit(data_iter.drop(columns = ["disciplina","situacao_categoria"]), data_iter["situacao_categoria"])
 
    #Extraindo informações do treino
    param_modelo = best_model.best_params_
    model_param_disciplinas[disciplina] = param_modelo['clf']
    best_acc_disciplinas[disciplina] = best_model.best_score_

    #Resultado de metricas
    result = pd.DataFrame(best_model.cv_results_)
    result = result[['mean_train_Accuracy', 'std_train_Accuracy','mean_test_Accuracy', 'std_test_Accuracy','rank_test_Accuracy']].copy()
    result["std_ratio"] = result.std_test_Accuracy/result.std_train_Accuracy
    result = result.sort_values(by="rank_test_Accuracy",ascending=True)    
    scoring_disciplinas[disciplina] = result.iloc[0,:]

    #Características mais importantes
    #feature_importances_disciplinas[disciplina] = grid.best_estimator_.feature_importances_
    
    #Salvando modelo
    joblib.dump(best_model, 'modelo_' + disciplina + '.pkl')


  #Transformando o dicionário para Dataframe
  df_best_acc = pd.DataFrame.from_dict(best_acc_disciplinas, orient="index")
  df_best_acc.columns = ["accuracy"]
  df_scor_disc = pd.DataFrame.from_dict(scoring_disciplinas, orient="index")

  #Concatenando DataFrames
  df_relatorio = pd.concat([df_best_acc, df_scor_disc], axis=1, sort=False)

  return (df_relatorio,model_param_disciplinas)

##4. Leitura da Base
<a id='leiturabase'></a>

In [0]:
df = pd.read_csv('')
df.head()

#Parte III - Desenvolvimento
<a id='enten_dados'></a>


## 1. Entendimento de Negócio
<a id='negocio'></a>

#### ● Determinar objetivos de Negócio
#### ● Estabelecer Critérios de Sucesso da Iniciativa
#### ● Análise do contexto: terminologia, benefício, riscos e plano de contingência
#### ● Determinar objetivos de Mineração de Dados
#### ● Plano do Projeto


## 2. Entendimento de Dados
<a id='enten_dados'></a>

### [2.1 Descrição de Dados](#descricao)
### [2.2 Qualidade de Dados](#qualidade)
### [2.3 Exploração de Dados](#exploracao)

###2.1 Descrição de Dados
<a id='descricao'></a>


#### Número de Atributos (colunas): x 
#### Número de Registros (linhas): y registros 

Base contêm y dados sem nenhum valor não-nulo. 

#### <center>Tipos dos Atributos:
| Inteiros | Floats(Ponto Flutuante) | Booleano (Binários) | String (Texto) |
| --- | --- | --- | --- |
| a | b | c | d |




####Descrição de Colunas:

<font color='blue'> <b>1. Nomes</b> </font> Contêm nomes de clientes

#### Coluna Alvo
<font color='red'> <b>10. Coluna_Preditora</b> </font> Coluna que quero predizer




In [0]:
df.info()

In [0]:
df.select_dtypes(include=['int64','float64']).describe()

In [0]:
profile = ProfileReport(df, title='Pandas Profiling Report', html={'style':{'full_width':True}})

###2.2 Qualidade dos dados
<a id='qualidade'></a>

Plotando correlação entre os dados númericos

In [0]:
corr = df.corr()
corr = corr.style.background_gradient(cmap='Blues')
corr

Falar um pouco da correlação

###2.3 Exploração de Dados
<a id='exploracao'></a>

##3. Preparação de Dados
<a id='preparo'></a>

##4. Métricas de Avaliação
<a id='metricas'></a>


#### <center>Métricas de Avaliação:
| Regressores | Classificadores
| --- | --- |
| <center>RMSE| <center>Matriz de Confusão |
| <center>MAE| <center>Acurácia |
| <center>R2| <center>Precisão / Recall / F1 |
| <center>R2 Ajustado| <center>AUC |




##5. Modelagem
<a id='#modelagem'></a>

Dicionário com hiper-parâmetros dos algoritmos preditivos

In [0]:
hyperparameters = [
                  {"clf":[RandomForestClassifier()],
                  "clf__n_estimators": [100],
                  "clf__criterion": ["entropy"],
                  "clf__max_leaf_nodes": [64],
                  "clf__random_state": [42],
                  "fs__score_func":[chi2],
                  "fs__k":[4,9,15,30]
                  },
              
                  {"clf":[KNeighborsClassifier()],
                  "clf__n_neighbors":[5,9,11],
                  "fs__score_func":[chi2],
                  "fs__k":[4,9,15,30]                 
                  },
              
                  {"clf":[SVC()],
                  "clf__kernel":["sigmoid",'rbf'],
                  "clf__degree":[3,4],
                  "clf__gamma":[0.1,0.5,1],
                  "clf__C":[0.001,1,2],
                  "fs__score_func":[chi2],
                  "fs__k":[4,9,15,25,31] 
                  },
              
                  {"clf":[GaussianNB()]
                  },
              
                  {"clf":[MLPClassifier()],
                  "clf__hidden_layer_sizes": [(64,),(128,)],
                  "clf__activation": ["logistic"],
                  "clf__solver": ["sgd"],
                  "clf__max_iter": [500],
                  "clf__early_stopping":[True],
                  "clf__n_iter_no_change":[20],
                  "clf__validation_fraction":[0.20], 
                  },
              
                  {"clf":[XGBClassifier()],
                  "clf__n_estimators": [50,100],
                  "clf__max_depth": [4,6],
                  "clf__learning_rate": [0.001, 0.01,0.1],
                  "clf__random_state": [42],
                  "clf__subsample": [1.0],
                  "clf__colsample_bytree": [1.0],
                  "fs__score_func":[chi2],
                  "fs__k":[5,8,15,25,31]
                  }
  ]

##6. Avaliação dos Modelos
<a id='avaliacao'></a>

### [6.1 Chamada da Função](#chamada)
### [6.2 Teste de Modelo](#teste)

###6.1 Chamada da Função
<a id='chamada'></a>

In [1]:
(df_relatorio, model_param_disciplinas) = avaliar_modelos(df, hyperparameters)

NameError: ignored

In [0]:
df_relatorio.head()

In [0]:
model_param_disciplinas

###6.2 Teste de Modelo
<a id='teste'></a>

Importando Modelo

In [0]:
modelo_teste = joblib.load('modelo_.pkl')

Aplicando a transformação no teste

In [0]:
num_transf = NumericalTransformer()
df_teste = model.transform(df)

In [0]:
df_teste.head(2)

Realizando teste de um modelo, prevendo probabilidade de ser aprovado

In [0]:
df_teste.iloc[42,:]

In [0]:
df_teste.info()

In [0]:
modelo_teste.predict(df_teste.iloc[42,:])

##7. Conclusão e Recomendações
<a id='conclusao'></a>

##8. Implantação de Modelos
<a id='implantacao'></a>